###  Хакатон

In [4]:
import pandas as pd
from catboost import Pool, CatBoostClassifier
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split 

%matplotlib inline

In [5]:
data_path = "../data/"
file_data = "labeled.csv"
file_data = os.path.join(data_path, file_data)

In [6]:
data = pd.read_csv(file_data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


In [7]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [8]:
data.toxic.value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [9]:
df_train, df_test = train_test_split(
    data,
    test_size=0.2, 
    stratify=data["toxic"],
    random_state=42
)


In [10]:
df_train.shape, df_test.shape

((11529, 2), (2883, 2))

In [11]:
df_train.toxic.value_counts(), df_test.toxic.value_counts()

(0.0    7668
 1.0    3861
 Name: toxic, dtype: int64,
 0.0    1918
 1.0     965
 Name: toxic, dtype: int64)

In [12]:
df_train.to_csv(os.path.join(data_path, "train.csv"), index=0)
df_test.to_csv(os.path.join(data_path, "test.csv"), index=0)

### Обучим

In [13]:
train = pd.read_csv(os.path.join(data_path, "train.csv"))
train.head()

,comment,toxic
0,Вы читали закон о банкротстве? Что бы объявить...,0.0
1,а у сяоми это постоянно,0.0
2,"Хотел занять? Тогда да,проще позвонить , чем у...",1.0
3,"заработки были без работы даже небольшие, но б...",0.0
4,Езжу каждый день. За этот год почти 30 тыс.\n,0.0


In [16]:
def fit_catboost(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    catboost_params = {},
    verbose = 100
):
    learn_pool = Pool(
        X_train, 
        y_train, 
        text_features=["text"], 
        feature_names=["text"]
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        text_features=["text"],
        feature_names=["text"]
    )
    catboost_default_params = {
        'iterations': 5000,
        'learning_rate': 0.05,
        'eval_metric': 'F1',
        'task_type': 'GPU',
        'use_best_model': True
    }
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)
    return model



In [18]:
X_train, X_val, y_train, y_val = train_test_split(
    train[["comment"]],
    train["toxic"],
    test_size=0.3, 
    stratify=train["toxic"],
    random_state=42
)
cat_boost_model = fit_catboost(X_train, X_val, y_train, y_val)

0:	learn: 0.5280431	test: 0.5305136	best: 0.5305136 (0)	total: 36ms	remaining: 2m 59s
100:	learn: 0.7228871	test: 0.7451670	best: 0.7477329 (1)	total: 2.51s	remaining: 2m 1s
200:	learn: 0.7294821	test: 0.7457181	best: 0.7477329 (1)	total: 4.71s	remaining: 1m 52s
300:	learn: 0.7341062	test: 0.7460317	best: 0.7477329 (1)	total: 7.08s	remaining: 1m 50s
400:	learn: 0.7412117	test: 0.7449250	best: 0.7477329 (1)	total: 9.65s	remaining: 1m 50s
500:	learn: 0.7460169	test: 0.7446996	best: 0.7477329 (1)	total: 12.2s	remaining: 1m 49s
600:	learn: 0.7522556	test: 0.7435897	best: 0.7477329 (1)	total: 14.5s	remaining: 1m 46s
700:	learn: 0.7568988	test: 0.7440213	best: 0.7477329 (1)	total: 17s	remaining: 1m 43s
800:	learn: 0.7609756	test: 0.7418925	best: 0.7477329 (1)	total: 19.2s	remaining: 1m 40s
900:	learn: 0.7659334	test: 0.7396529	best: 0.7477329 (1)	total: 21.5s	remaining: 1m 37s
1000:	learn: 0.7703592	test: 0.7387628	best: 0.7477329 (1)	total: 23.8s	remaining: 1m 35s
1100:	learn: 0.7755639	tes

In [17]:
cat_boost_model.save_model("hack_model")